In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from config import DOWNLOAD_DIR
from pathlib import Path


### Merge the datasets

We need to merge the three (sometimes two) datasets into a single dataframe.

total_precipitation_current_month.nc total_precipitation_previous_year.nc
total_precipitation_current_year.nc


In [2]:
prior_year = xr.open_dataset(Path(DOWNLOAD_DIR) / "inputs" / "total_precipitation_previous_year.nc")
current_year = xr.open_dataset(Path(DOWNLOAD_DIR) / "inputs" / "total_precipitation_current_year.nc")
current_month = xr.open_dataset(Path(DOWNLOAD_DIR) / "inputs" / "total_precipitation_current_month.nc")

In [3]:
merged = xr.merge([prior_year, current_year, current_month])

In [4]:
merged


<xarray.Dataset>
Dimensions:    (time: 14904, longitude: 221, latitude: 129, expver: 2)
Coordinates:
  * time       (time) datetime64[ns] 2021-01-01 ... 2022-09-13T23:00:00
  * longitude  (longitude) float32 -180.0 -179.8 -179.5 ... -125.5 -125.2 -125.0
  * latitude   (latitude) float32 76.0 75.75 75.5 75.25 ... 44.5 44.25 44.0
  * expver     (expver) int32 1 5
Data variables:
    tp         (time, latitude, longitude, expver) float32 2.03e-06 ... 0.0
Attributes:
    Conventions:  CF-1.6
    history:      2022-09-20 00:43:27 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...